In [0]:
# =========================================================
# 📦 LAB 3
# =========================================================

# 1. SETUP & CONFIGURATION
# =========================================================
spark.conf.set(
    "fs.azure.account.key.goodreads60104699.dfs.core.windows.net",
    "slStfm+E+Qmq6wT2Yzk4kxaXe16kLK5e+erlt72LzutDZGHdWjDkvMYebsSBOnV2JH0NCnIjLj40+AStdSiXVA=="
)

# Test connection
display(dbutils.fs.ls("abfss://lakehouse@goodreads60104699.dfs.core.windows.net/"))

# 2. LOAD SILVER LAYER DATA
# =========================================================
books = spark.read.parquet("abfss://lakehouse@goodreads60104699.dfs.core.windows.net/silver/books/")
authors = spark.read.parquet("abfss://lakehouse@goodreads60104699.dfs.core.windows.net/silver/authors/")
reviews = spark.read.parquet("abfss://lakehouse@goodreads60104699.dfs.core.windows.net/silver/reviews/")

print("✅ Silver layer data loaded:")
print(f"Books: {books.count():,} rows")
print(f"Authors: {authors.count():,} rows") 
print(f"Reviews: {reviews.count():,} rows")

books.show(5, truncate=False)
reviews.show(5, truncate=False)

# 3. CLEAN REVIEWS DATASET
# =========================================================
from pyspark.sql.functions import col, trim, length, to_timestamp, coalesce, lit

# Clean reviews step-by-step
reviews_clean = reviews.filter(
    col("review_id").isNotNull() &
    col("book_id").isNotNull() &
    col("user_id").isNotNull()
)

# Validate ratings (1-5 only)
reviews_clean = reviews_clean.withColumn("rating_int", col("rating").cast("int")).filter(
    col("rating_int").isNotNull() &
    (col("rating_int") >= 1) &
    (col("rating_int") <= 5)
)

# Clean text and filter short reviews
reviews_clean = reviews_clean.withColumn("review_text", trim(col("review_text"))).filter(
    col("review_text").isNotNull() &
    (length(col("review_text")) >= 10)
)

# Remove duplicates
reviews_clean = reviews_clean.dropDuplicates(["review_id"])

# Clean and filter dates
reviews_clean = reviews_clean.withColumn(
    "date_added",
    coalesce(
        to_timestamp(col("date_added"), "yyyy-MM-dd HH:mm:ss"),
        to_timestamp(col("date_added"), "yyyy-MM-dd"),
        to_timestamp(col("date_added"), "MM/dd/yyyy")
    )
).filter(col("date_added") >= lit("2014-01-01"))

# Final selection
reviews_clean = reviews_clean.select(
    "review_id", "book_id", "user_id",
    col("rating_int").alias("rating"),
    "review_text", "date_added"
)

print(f"✅ Cleaned reviews: {reviews_clean.count():,} rows")
reviews_clean.show(5, truncate=False)

# 4. SAVE CLEANED REVIEWS
# =========================================================
output_path = "abfss://lakehouse@goodreads60104699.dfs.core.windows.net/processed/reviews_clean_final/"

# Clear old data
spark.catalog.clearCache()
dbutils.fs.rm(output_path, recurse=True)

# Save cleaned reviews
reviews_clean.write.mode("overwrite").parquet(output_path)
print("✅ Cleaned reviews saved to processed layer")

# 5. BUILD GOLD LAYER - CURATED REVIEWS
# =========================================================
from pyspark.sql.functions import explode, split, regexp_replace, monotonically_increasing_id

# Create author bridge table
authors_bridge = (
    books
    .select(col("book_id"), explode(split(col("authors"), ",")).alias("name"))
    .withColumn("name", trim(regexp_replace(col("name"), r"[\[\]']", "")))
    .filter(col("name") != "")
    .withColumn("author_id", monotonically_increasing_id().cast("string"))
    .dropDuplicates(["book_id", "author_id"])
)

# Join books with authors
books_authors = books.join(authors_bridge, on="book_id", how="inner")

# Create curated reviews by joining all datasets
curated_reviews = (
    reviews_clean
    .join(books_authors, on="book_id", how="inner")
    .select(
        "review_id", "book_id", "title",
        "author_id", "name", "user_id",
        "rating", "review_text", "date_added", "average_rating"
    )
)

print("✅ Gold layer curated reviews created")
curated_reviews.show(5, truncate=False)

# 6. FEATURE ENGINEERING
# =========================================================
from pyspark.sql.functions import regexp_replace, when, size, split, avg, count

# Clean review text and calculate length
curated_reviews = curated_reviews.withColumn(
    "review_text",
    regexp_replace(col("review_text"), r"[\n\r\t\u00A0]", " ")
).withColumn("review_text", trim(col("review_text")))

curated_reviews = curated_reviews.withColumn(
    "review_length",
    when(col("review_text").isNotNull(), size(split(col("review_text"), r"\s+"))).otherwise(0)
)

# Compute aggregations
avg_rating_book = curated_reviews.groupBy("book_id").agg(avg("rating").alias("avg_rating_book"))
review_count_book = curated_reviews.groupBy("book_id").agg(count("review_id").alias("review_count_book"))
avg_rating_author = curated_reviews.groupBy("name").agg(avg("rating").alias("avg_rating_author"))

# Join aggregations
curated_final = (
    curated_reviews
    .join(avg_rating_book, "book_id", "left")
    .join(review_count_book, "book_id", "left")
    .join(avg_rating_author, "name", "left")
)

print("✅ Feature engineering completed")
curated_final.show(5, truncate=False)

# 7. SAVE GOLD LAYER & REGISTER TABLE
# =========================================================
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

# Remove old data
gold_path = "abfss://lakehouse@goodreads60104699.dfs.core.windows.net/gold/curated_reviews/"
dbutils.fs.rm(gold_path, recurse=True)

# Save as Delta table
curated_final.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(gold_path)

# Register as table
curated_final.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("curated_reviews")

print("🏆 Gold layer saved and registered as Delta table!")

# 8. VALIDATION & ANALYSIS
# =========================================================
print("📊 FINAL VALIDATION:")

# Show tables
spark.sql("SHOW TABLES").show()

# Summary statistics
spark.sql("""
SELECT
    COUNT(DISTINCT book_id) AS total_books,
    COUNT(DISTINCT author_id) AS total_authors, 
    COUNT(DISTINCT user_id) AS total_users,
    ROUND(AVG(rating), 2) AS avg_rating,
    MIN(rating) AS min_rating,
    MAX(rating) AS max_rating
FROM curated_reviews
""").show()

# Top authors
spark.sql("""
SELECT name AS author_name, COUNT(*) AS review_count
FROM curated_reviews
GROUP BY author_name
ORDER BY review_count DESC
LIMIT 5
""").show(truncate=False)

# Final count
spark.sql("SELECT COUNT(*) AS total_reviews FROM curated_reviews").show()

print("🎉 LAB 3 COMPLETED ")

path name size modificationTime abfss://lakehouse@goodreads60104699.dfs.core.windows.net/bronze/ bronze/ 0 1762769150000 abfss://lakehouse@goodreads60104699.dfs.core.windows.net/gold/ gold/ 0 1762186237000 abfss://lakehouse@goodreads60104699.dfs.core.windows.net/processed/ processed/ 0 1762183898000 abfss://lakehouse@goodreads60104699.dfs.core.windows.net/raw/ raw/ 0 0 abfss://lakehouse@goodreads60104699.dfs.core.windows.net/silver/ silver/ 0 1762158061000

✅ Silver layer data loaded:
Books: 2,360,655 rows
Authors: 836,709 rows
Reviews: 8,581,702 rows
+--------+----------------------------------------------+-----------------------------------------------+--------------+
|book_id |title                                         |authors                                        |average_rating|
+--------+----------------------------------------------+-----------------------------------------------+--------------+
|6767112 |America's Cold War: The Politics of Insecurity|[{67986, }, {1910, }]                          |4.2           |
|9464370 |March Story, Volume 2 (March Story #2)        |[{4851127, }, {4714037, Illustrator}]          |4.37          |
|1605088 |The Flying Tortoise: An Igbo Tale             |[{196098, Retelling}, {2877625, Illustrations}]|3.73          |
|31159191|Hard-Hearted Highlander (Highland Grooms, #3) |[{49377, }]                                    |3.68          |
|198819  |The Children of Men                           |